In [1]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from implicit.bpr import BayesianPersonalizedRanking
import recometrics
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import recmetrics
import tqdm


In [22]:
train_df = pd.read_csv("netflixFullTrain.csv",sep=",")
test_df = pd.read_csv("netflixFullTest.csv",sep=",")


In [23]:
users = test_df['user_id'].unique()
items = test_df['movie_id'].unique()

In [ ]:
train_df = train_df[train_df['user_id'].isin(users)]
train_df = train_df[train_df['movie_id'].isin(items)]
# some how the test and train split was sparse enough i guess that some of the few rows/ columns
# that had value were removed due to the split
#https://stackoverflow.com/questions/27965295/dropping-rows-from-dataframe-based-on-a-not-in-condition


In [5]:
print(len(users),len(items))

393147 17622


In [6]:
user_positives = test_df.groupby('user_id')['movie_id'].apply(set).to_dict()


In [7]:

test_ratings_matrix = test_df.pivot(index='user_id', columns='movie_id', values='rating')


C:\Users\Owner\AppData\Local\Temp\ipykernel_34880\2094183977.py:1: PerformanceWarning: The following operation may generate 6928036434 cells in the resulting pandas object.
  test_ratings_matrix = test_df.pivot(index='user_id', columns='movie_id', values='rating')


In [21]:
display(test_ratings_matrix)

movie_id,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user_id,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
del test_df

In [24]:
train_df["rating"] =1
# train_df =train_df.drop("tstamp",axis=1)
train_df =train_df.drop("Unnamed: 0",axis=1)

train_df.columns =["UserId", "ItemId", "Count"]

display(train_df)


,UserId,ItemId,Count
0,505016,5082,1
1,515243,17506,1
2,2454190,5963,1
3,453742,5862,1
4,460424,14691,1
...,...,...,...
7033630,2604861,11283,1
7033631,1086968,6205,1
7033632,1227848,9695,1
7033633,1847157,10928,1


In [39]:
modelPF = PoisMF(reindex=True, method="tncg", use_float=True,
                    early_stop=False, reuse_prev=True,
                    k=20, niter=10, maxupd=200, l2_reg=1e3)\
                .fit(coo)

In [41]:
display(modelPF.A.shape) # user
display(modelPF.B.shape) # items


(445077, 20)

(17760, 20)

In [27]:
pred = np.matmul(modelPF.A,modelPF.B.T)

In [28]:
display(pred)

array([[8.9819914e-05, 6.8380707e-03, 0.0000000e+00, ..., 0.0000000e+00,
        2.1861300e-07, 3.0304398e-07],
       [0.0000000e+00, 7.7545717e-03, 1.3245443e-03, ..., 3.5621861e-06,
        1.1999547e-06, 7.9486557e-07],
       [0.0000000e+00, 7.1805944e-03, 1.2436036e-03, ..., 1.2105355e-06,
        2.2984141e-06, 2.7344356e-06],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0942958e-07, 7.2342239e-08],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        2.4711406e-07, 1.6664049e-07],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0897341e-07, 7.2040677e-08]], dtype=float32)

In [29]:
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [31]:
display(pred.shape)

(445077, 17760)

Delete unused variables to free up space

In [18]:
del users
del items

del modelPF
del train_df


In [19]:
predv = torch.from_numpy(pred)
#https://medium.com/@heyamit10/converting-a-numpy-array-to-a-tensor-step-by-step-df329c44b035

In [20]:
del pred

In [ ]:
marData = predv.detach().numpy()
marDF = pd.DataFrame(marData)
marDF.index.name = "user"
marDF.columns.name = "items"

# marDF = marDF.stack().reset_index()
# marDF.columns = ['user', 'item', 'score']
# print("MAR",calculate_metrics(marDF,user_positives))


In [ ]:
def calculate_metrics_matrix(pred_df, user_positives, k_list=[10, 50, 100]):
    metrics = {
        'MAR@ALL': [],
        'MAR@1000': [],
    }

    user_ids = pred_df.index.values  # Get actual user identifiers

    for user in tqdm.tqdm(user_ids, desc="Processing Users"):
        positives = user_positives.get(user, set())
        if len(positives) == 0:
            continue

        data = pred_df.loc[user].values  
        # get descending order like original code
        #https://www.w3schools.com/python/numpy/numpy_array_sort.asp
        placeholder = np.sort(data)
        #https://www.geeksforgeeks.org/python-reverse-a-numpy-array/
        all_items = np.flip(placeholder)
                
        relevant_ranks1k = []
        relevant_ranks_all=[]
        for rank, item in enumerate(all_items[:1000], start=1):
            if item in positives:
                relevant_ranks1k.append(rank)
        #calculate relevant
        for rank, item in enumerate(all_items, start=1):
            if item in positives:
                relevant_ranks_all.append(rank)



        if relevant_ranks_all:
            metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
        if relevant_ranks1k:
            metrics['MAR@1000'].append(np.mean(relevant_ranks1k))
        

    
    final = {
        'MAR@ALL': np.mean(metrics['MAR@ALL']),
        'MAR@1000': np.mean(metrics['MAR@1000']),
    }

    return final

In [ ]:
calculate_metrics_matrix(marDF,user_positives)

In [ ]:
del marDF
del user_positives

In [ ]:
true = torch.tensor(test_ratings_matrix.values,dtype=torch.float16)

In [ ]:
del test_ratings_matrix

In [ ]:
display(true)
display(predv)

In [ ]:
true = true.nan_to_num(0)
predv = predv.nan_to_num(0)

In [ ]:
display(true.shape)
display(predv.shape)


In [ ]:
true[true<0]=0
true[true>1]=1


In [ ]:
def metricStuff(pred,true):
    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))
    # code breaks when trying to compute metrics that are not @k
    # I.e. not enough memory too compute and not enough storage space on hard drive to write to
    # print("MRR@all",mean_reciprocal_rank(pred, true))
    # print("MAP",mean_average_precision(pred, true))
    

In [ ]:
metricStuff(predv,true)